<a href="https://colab.research.google.com/github/TexasherScript/python-sql/blob/main/datadashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install shiny plotly pandas

In [3]:
from shiny import App, render, ui, reactive
import pandas as pd
import plotly.express as px

# Load and clean the dataset
def load_and_clean_data(filepath):
    df = pd.read_csv(filepath)

    # Convert FinancialYear to a start year
    df['FinancialYear_Start'] = df['FinancialYear'].str[:4].astype(int)

    # Convert numeric columns to integers (if they are stored as strings)
    numeric_columns = ['NumberOfAdmissions', 'NumberOfDischarges', 'NumberOfStays',
                       'NumberOfPatients', 'NumberOfHospitalResidents']
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert non-numeric to NaN

    # Drop rows with missing values in essential columns
    df = df.dropna(subset=numeric_columns)

    return df

# Load the cleaned data
data_filepath = "ba979207-fbd0-4f96-b855-ac7540e03b23.csv"  # Path to your CSV file
df = load_and_clean_data(data_filepath)

# Define UI
app_ui = ui.page_fluid(
    ui.h2("Hospital Data Dashboard"),

    # Dropdown for selecting Financial Year
    ui.input_select("year", "Select Financial Year:", choices=df['FinancialYear_Start'].unique(), selected=df['FinancialYear_Start'].min()),

    # Table to display data
    ui.output_table("data_table"),

    # Plot outputs
    ui.output_plot("admissions_plot"),
    ui.output_plot("stays_plot")
)

# Define server logic
def server(input, output, session):

    # Filter data based on selected year
    @reactive.Calc
    def filtered_data():
        return df[df['FinancialYear_Start'] == input.year()]

    # Display data table
    @output
    @render.table
    def data_table():
        return filtered_data()

    # Plot Admissions by Dataset
    @output
    @render.plot
    def admissions_plot():
        fig = px.bar(filtered_data(), x="Dataset", y="NumberOfAdmissions", color="Dataset",
                     title="Number of Admissions by Dataset",
                     labels={"NumberOfAdmissions": "Admissions"})
        fig.update_layout(showlegend=False)
        return fig

    # Plot Number of Stays vs. Number of Patients
    @output
    @render.plot
    def stays_plot():
        fig = px.scatter(filtered_data(), x="NumberOfStays", y="NumberOfPatients",
                         title="Number of Stays vs. Number of Patients",
                         labels={"NumberOfStays": "Stays", "NumberOfPatients": "Patients"})
        return fig

# Create the app
app = App(app_ui, server)

# Run the app within the Jupyter Notebook
from shinywidgets import render_widget, ShinyWidget

render_widget(app)


AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [4]:
# Create the app
app = App(app_ui, server)

# Run the app within the Jupyter Notebook
from shinywidgets import render_widget, ShinyWidget

render_widget(app)

NameError: name 'app_ui' is not defined